### Entity Matching

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import distance

def load_df(filename):
    return pd.read_csv(f'./datasets/{filename}.csv')

layout1 = load_df('ABC_layout_1')
layout2 = load_df('PQR_layout_2')
layout3 = load_df('layout_3_voters')
layout4 = load_df('KLM_layout_4')
layout5 = load_df('layout_5_license')

layout1 = layout1.rename(columns={"First Name": "Name", "Father Name": "Father_Name", "Permanent_Adress":"Permanent_Address"})
layout2 = layout2.rename(columns = {"Customer_ID": "Mobile Number"})
layout3 = layout3.rename(columns={"votersName": "Name", "votersFatherName": "Father_Name", "votersMotherName": "Mother Name", " Gender": "Gender", "Permanent_Adress":"Permanent_Address"})
layout4 = layout4.rename(columns={"Father Name": "Father_Name"})

def sanitize(df):
    return df.map(lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else '' if pd.isna(x) else x)

layouts = [layout1, layout2, layout3, layout4, layout5]
layout_copies = [layout.copy() for layout in layouts]

for i in range(len(layout_copies)):
    layout_copies[i] = sanitize(layout_copies[i])

def create_soup(df, df_, soup, soup_name):
    df[soup_name] = df_[soup].apply(lambda x: ' '.join(x.values.astype(str)).lower(), axis=1)

soup = ['Name', 'Date of Birth', 'Father_Name']

for i, j, k, in zip(layouts, layout_copies, range(len(layouts))):
    create_soup(i, j, soup, f"soup{k+1}")

def combine_layouts(A, B, soup_A, soup_B, threshold=0.3):

    tfidf = TfidfVectorizer(stop_words='english')
    
    combined_soup = pd.concat([A[soup_A], B[soup_B]], ignore_index=True)
    tfidf.fit(combined_soup)
    
    tfidf_matrix_A = tfidf.transform(A[soup_A])
    tfidf_matrix_B = tfidf.transform(B[soup_B])
    
    similarity = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)
    similarity_df = pd.DataFrame(similarity, index=A.index, columns=B.index)

    max_idx_row = similarity_df.idxmax(axis=1)
    similarity_mask = similarity_df.max(axis=1) > threshold

    # Initialize the combined DataFrame with all columns from both DataFrames
    combined_columns = list(set(A.columns) | set(B.columns))
    combined_data = pd.DataFrame(columns=combined_columns)
    
    # Merge the similar rows 
    for idx_A in A.index:
        if similarity_mask[idx_A]:
            idx_B = max_idx_row[idx_A]
            combined_row = A.loc[idx_A].combine_first(B.loc[idx_B])
        else:
            combined_row = A.loc[idx_A]
        combined_data = pd.concat([combined_data, combined_row.to_frame().T], ignore_index=True)
    
    # Append non-similar rows from B to A
    new_records = B.loc[~B.index.isin(max_idx_row[similarity_mask].values)]
    result = pd.concat([combined_data, new_records], ignore_index=True)
    result.drop(columns=soup_B, inplace=True)
    return result


result_12 = combine_layouts(layout1, layout2, 'soup1', 'soup2')
result_123 = combine_layouts(result_12, layout3, 'soup1', 'soup3')
result_1234 = combine_layouts(result_123, layout4, 'soup1', 'soup4')
final_result = combine_layouts(result_1234, layout5, 'soup1', 'soup5')
final_result.drop(columns='soup1', inplace=True)
final_result

,votersAge,Date of Birth,SpouseName,License Number,Customer Code,Customer ID,Temporary_Address,Gender,PAN_Number,SC Number,Blood Group,Father_Name,Permanent_Address,Mother Name,votersID,Name,Citizenship Number,Mobile Number,National Id
0,45,1990-01-01,Sita Thapa,15-05-58353205,21216874,3245,"Gongabu, Kathmandu, Nepal",Male,ABCDE1234F,001.01.01,AB+,Ram Bahadur Thapa,"Baluwatar, Kathmandu, Nepal",Laxmi Thapa,11116874,Ram Thapa,624-93227-32431/660086,1234567890,AB123C
1,38,1991-02-02,Ravi Sharma,21-08-00435579,22359363,3246,"New Road, Pokhara, Nepal",Female,FGHIJ5678K,001.01.02,AB-,Hari Prasad Shrestha,"Lakeside, Pokhara, Nepal",Radha Sharma,22259363,Sita Shrestha,747-42087-31417/584714,2345678901,DE456F
2,52,1992-03-03,Maya Adhikari,93-12-35351480,33485241,3247,"Pulchowk, Lalitpur, Nepal",Male,LMNOP9012L,001.01.03,B-,Gopal Krishna Gurung,"Chitwan National Park, Chitwan, Nepal",Gita Adhikari,33385241,Hari Gurung,389-45382-93886/821590,3456789012,GH789I
3,30,1993-04-04,Surya Rai,65-03-68139881,45475489,3248,"Bagbazar, Kathmandu, Nepal",Female,QRSTU3456M,001.01.04,A-,Shyam Lal Tamang,"Biratnagar, Morang, Nepal",Mina Rai,44475489,Gita Tamang,571-38785-99733/440035,4567890123,JK012L
4,27,1994-05-05,Sarita Karki,14-11-40056582,56562139,3249,"Balkumari, Lalitpur, Nepal",Male,VWXYZ7890N,001.01.05,A+,Krishna Raj Lama,"Bharatpur, Chitwan, Nepal",Kalpana Karki,55562139,Mohan Lama,864-17331-40021/961722,5678901234,MN345O
5,41,1995-06-06,Pemba Gurung,82-09-81734599,67698214,3250,"Chabahil, Kathmandu, Nepal",Female,ABCD1234PQ,001.01.06,B-,Narayan Kumar Magar,"Butwal, Rupandehi, Nepal",Nima Gurung,66698214,Radha Magar,033-51347-62581/380746,6789012345,PQ678R
6,36,1996-07-07,Laxmi Shrestha,26-03-50185868,78714635,3251,"Kumaripati, Lalitpur, Nepal",Male,EFGH5678RS,001.01.07,AB+,Govinda Bahadur Rai,"Hetauda, Makwanpur, Nepal",Saru Shrestha,77714635,Krishna Rai,068-37653-84341/852787,7890123456,ST901U
7,50,1997-08-08,Raj Maharjan,98-03-72394228,89847326,3252,"New Baneshwor, Kathmandu, Nepal",Female,TUVW9012XY,001.01.08,A-,Shiva Narayan Sherpa,"Janakpur, Dhanusa, Nepal",Nanu Maharjan,88847326,Sarita Sherpa,323-37869-95909/623481,8901234567,VW234X
8,43,1998-09-09,Pema Tamang,68-01-80388982,91953421,3253,"Boudha, Kathmandu, Nepal",Male,ZABC3456DE,001.01.09,B-,Bhagirath Bahadur Karki,"Nepalgunj, Banke, Nepal",Lhamu Tamang,99953421,Bikash Karki,406-94259-70142/249811,9012345678,YZ567A
9,34,1999-10-10,Manish KC,42-08-02614125,10203847,3254,"Kalanki, Kathmandu, Nepal",Female,FGHI7890JK,001.01.10,A-,Surya Bahadur Bhandari,"Dharan, Sunsari, Nepal",Lila KC,10103847,Nisha Bhandari,272-28301-42325/881177,123456789,BC890D
